In [ ]:
#
# Attempts to implement the following:
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from shapely import Polygon
from   udist_scatterplots_via_sectors import UDistScatterPlotsViaSectors, xyUniformSampleDistributionSectorTransformDEBUG, xyUniformSampleDistributionSectorTransform
import time
import rtsvg
rt = rtsvg.RACETrack()
import random
num_of_pts   = [100, 200, 50]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])

_iterations_ = 4

x_new, y_new, svgs, svgs_for_sectors, svg_animation, df_fine = xyUniformSampleDistributionSectorTransformDEBUG(rt, _xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_)

t0 = time.time()
xyUniformSampleDistributionSectorTransform(rt, _xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_)
t1 = time.time()
udspvs = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_, debug=True)
t2 = time.time()
#
# Comparison (w/ fixed implementation)
# 351 Points (128 Iterations), 18.74s Reference Time, 1.66s Polars Time | (M1 Pro 16G) 
# 351 Points (256 Iterations), 38.45s Reference Time, 7.28s Polars Time | (M1 Pro 16G) 
#
print(f'{len(_xvals_)} Points ({_iterations_} Iterations), {t1-t0:.2f}s Reference Time, {t2-t1:.2f}s Polars Time')
rt.tile([svg_animation, udspvs.animateIterations(animation_dur="4s")], spacer=10)

In [ ]:
# 2048 Iterations w/ 351 Points | M1 Pro 16G
#
#                 sector_sums | 3.959
#                 arctangents | 2.436
#   ray_segment_intersections | 2.289
#              explode_points | 2.251
#         add_missing_sectors | 1.839
#                   area_calc | 1.254
#         sector_uv_summation | 0.907
#                point_update | 0.784
#       prepare_sector_angles | 0.659
#          join_sector_angles | 0.592
#                 all_sectors | 0.421
#                   normalize | 0.269
#                  prepare_df | 0.000
udspvs = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, iterations=16)
_lu_ = {'subroutine':[], 'time':[]}
for k in udspvs.time_lu: _lu_['subroutine'].append(k), _lu_['time'].append(udspvs.time_lu[k])
_df_ = pl.DataFrame(_lu_).sort(by='time', descending=True)
for i in range(len(_df_['subroutine'])): print(f'{_df_["subroutine"][i]:>28} | {_df_["time"][i]:.3f}')

In [ ]:
import copy
timing   = {'pts':[], 'iters':[], 'time':[]}
time_lus = []
for num_pts in [10,20,30]: # [1000, 1500, 2000]:
    for num_iter in [4, 8, 12]: # [16, 32, 64, 128, 256]:
        _xvals_, _yvals_ = [], []
        for i in range(num_pts):
            x, y = random.random() * 100, random.random() * 100
            _xvals_.append(x), _yvals_.append(y)
        t0 = time.time()
        udspvs = UDistScatterPlotsViaSectors(_xvals_, _yvals_, iterations=num_iter)
        t1 = time.time()
        print(f'{num_pts:>5} Pts | {num_iter:>3} Iters | {t1-t0:.2f}s')
        timing['pts'].append(num_pts), timing['iters'].append(num_iter), timing['time'].append(t1-t0)
        time_lus.append(copy.deepcopy(udspvs.time_lu))
#rt.xy(pl.DataFrame(timing), x_field='iters', y_field='time', line_groupby_field='pts', color_by='pts') #._repr_svg_()

'<svg id="xy_13657337" x="0" y="0" width="256" height="256" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="255" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 239 L 17 3 L 251 3 L 251 239 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,237 32,233 63,226 126,213 251,185 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,231 32,223 63,207 126,174 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,225 32,209 63,181 126,123 251,3 " stroke="#72b1b4" stroke-width="1" fill="none" /><circle  cx="32" cy="209" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="181" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="185" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="231" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="223" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="226" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="233" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="3" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="174" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="237" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="213" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="225" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="207" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="123" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="253" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="253" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="237" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,237)">0.5</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">36.0</text><text x="13" text-anchor="middle" y="120.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,120.0)">time</text><rect width="255" height="256" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg>'

In [ ]:
#
# performance issues (in order): sector_sums ... then explode_points ... then arctangents
#
_lu_ = {'pts':[], 'iters':[], 'subroutine':[], 'subroutine_time':[]}
for i in range(len(time_lus)):
    _time_lu_               = time_lus[i]
    _pts_, _iters_, _times_ = timing['pts'][i], timing['iters'][i], timing['time'][i]
    for k in _time_lu_:
        _lu_['pts'].append(_pts_), _lu_['iters'].append(_iters_), _lu_['subroutine'].append(k), _lu_['subroutine_time'].append(_time_lu_[k])

#rt.smallMultiples(pl.DataFrame(_lu_), category_by='subroutine', sm_type='xy', y_axis_independent=False,
#                  sm_params={'x_field':'iters', 'y_field':'subroutine_time', 'line_groupby_field':'pts', 'color_by':'pts'},
#                  sort_by='field', sort_by_field='subroutine_time',
#                  h_sm_override=128, w_sm_override=256, w=1300) # ._repr_svg_()

'<svg id="smallMultiples_21023" x="0" y="0" width="1292" height="438" xmlns="http://www.w3.org/2000/svg"><rect width="1291" height="437" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><svg id="smallMultiples_21023_0" x="2" y="2" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,105 32,102 63,95 126,82 251,56 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,107 32,106 63,103 126,99 251,88 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,102 32,95 63,82 126,56 251,3 " stroke="#72b1b4" stroke-width="1" fill="none" /><circle  cx="32" cy="95" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="99" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="106" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="3" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="103" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="102" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="82" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="56" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="107" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="56" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="88" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="105" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="95" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="102" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="82" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="142" font-family="Times" fill="#000000" font-size="14px">sector_sums</text><svg id="smallMultiples_21023_1" x="260" y="2" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,106 32,103 63,97 126,85 251,59 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,107 126,105 251,101 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,107 32,105 63,102 126,94 251,80 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="32" cy="108" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="103" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="97" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="105" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="106" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="80" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="107" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="105" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="85" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="101" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="94" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="107" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="59" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="102" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="142" font-family="Times" fill="#000000" font-size="14px">explode_points</text><svg id="smallMultiples_21023_2" x="518" y="2" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,105 32,103 63,98 126,88 251,68 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,107 63,106 126,103 251,97 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,107 32,105 63,102 126,97 251,85 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="251" cy="85" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="88" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="97" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="68" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="103" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="103" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="107" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="105" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="98" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="102" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="97" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="106" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="107" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="105" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="142" font-family="Times" fill="#000000" font-size="14px">arctangents</text><svg id="smallMultiples_21023_3" x="776" y="2" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,106 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,105 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,106 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="251" cy="105" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="106" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="107" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="142" font-family="Times" fill="#000000" font-size="14px">add_missing_sectors</text><svg id="smallMultiples_21023_4" x="1034" y="2" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,106 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,106 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,106 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="106" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="107" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="142" font-family="Times" fill="#000000" font-size="14px">ray_segment_intersections</text><svg id="smallMultiples_21023_5" x="2" y="148" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#b3c79f" stroke-width="1" fill="none" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="107" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="288" font-family="Times" fill="#000000" font-size="14px">area_calc</text><svg id="smallMultiples_21023_6" x="260" y="148" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="107" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="288" font-family="Times" fill="#000000" font-size="14px">sector_uv_summation</text><svg id="smallMultiples_21023_7" x="518" y="148" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="288" font-family="Times" fill="#000000" font-size="14px">point_update</text><svg id="smallMultiples_21023_8" x="776" y="148" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="288" font-family="Times" fill="#000000" font-size="14px">join_sector_angles</text><svg id="smallMultiples_21023_9" x="1034" y="148" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="288" font-family="Times" fill="#000000" font-size="14px">prepare_sector_angles</text><svg id="smallMultiples_21023_10" x="2" y="294" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="434" font-family="Times" fill="#000000" font-size="14px">all_sectors</text><svg id="smallMultiples_21023_11" x="260" y="294" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="434" font-family="Times" fill="#000000" font-size="14px">normalize</text><svg id="smallMultiples_21023_12" x="518" y="294" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,109 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="109" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="434" font-family="Times" fill="#000000" font-size="14px">prepare_df</text><rect width="1291" height="437" x="0" y="0" fill="none" fill-opacity="0.0" stroke="#000000" /></svg>'

In [ ]:
#
# Prototype for Grouping Points into Tiles...
# ... then using those for sector_sums, explode_points, and arctangents
#
num_of_tiles = 64

#num_of_pts   = [100_000, 200_000, 50_000]
num_of_pts   = [1000, 2000, 500]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])

df = pl.DataFrame({'x':_xvals_, 'y':_yvals_, 'w':_weights_, 'c':_colors_}).with_row_index('__index__')
df = df.with_columns((0.02 + 0.96 * (pl.col('x') - pl.col('x').min())/(pl.col('x').max() - pl.col('x').min())).alias('x'), 
                     (0.02 + 0.96 * (pl.col('y') - pl.col('y').min())/(pl.col('y').max() - pl.col('y').min())).alias('y'))

# Put into tiles -- xi, yi
t = time.time()
df = df.with_columns((pl.col('x') * num_of_tiles).cast(pl.Int16).alias('xi'), (pl.col('y') * num_of_tiles).cast(pl.Int16).alias('yi'))
t_tiling = time.time() - t

# Sum those tiles ... requires a new dataframe because we'll need the original points later on...
t = time.time()
df_tiled = df.group_by(['xi','yi']).agg(pl.col('w').sum().alias('_tile_sum_'))
t_tile_sums = time.time() - t

print(f'{t_tiling=:.3f} | {t_tile_sums=:.3f}')

df

In [ ]:
# tile_to_rect[(32,32)]
# (0.5, 0.5, 0.515625, 0.515625)
_render_tiles_, _render_points_, _render_rays_ = True, True, True
xpt, ypt = round(0.5 + (0.515625 - 0.5) * random.random(),6), round(0.5 + (0.515625 - 0.5) * random.random(),6) 
xpt, ypt = 0.5+10e-4,0.5+10e-4

svg = [f'<svg x="0" y="0" width="1024" height="1024" viewBox="0.0 0.0 1.0 1.0">']
svg.append(f'<rect x="0" y="0" width="1.0" height="1.0" fill="#ffffff"/>')

tile_to_rect = {}
if _render_tiles_:
    for xi in range(num_of_tiles):
        x0, x1 = xi/float(num_of_tiles), (xi+1)/float(num_of_tiles)
        for yi in range(num_of_tiles):
            y0, y1 = yi/float(num_of_tiles), (yi+1)/float(num_of_tiles)
            tile_to_rect[(xi,yi)] = (x0, y0, x1, y1)
            _color_   = rt.co_mgr.getColor(str((xi,yi)))
            svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="{_color_}" fill-opacity="0.5" />')

if _render_points_:
    for i in range(len(df)):
        x,y,xi,yi = df['x'][i], df['y'][i], df['xi'][i], df['yi'][i]
        _color_   = rt.co_mgr.getColor(str((xi,yi)))
        svg.append(f'<circle cx="{x}" cy="{y}" r="0.001" fill="{_color_}"/>')

tiles_intersected_by_rays = set()
if _render_rays_:
    for _sector_ in range(16):
        a0, a1         = _sector_*2*pi/16, (_sector_+1)*2*pi/16
        u0, v0, u1, v1 = cos(a0), sin(a0), cos(a1), sin(a1)
        svg.append(f'<line x1="{xpt}" y1="{ypt}" x2="{xpt+u0*2.0}" y2="{ypt+v0*2.0}" stroke="#000000" stroke-width="0.0001"/>')
        svg.append(f'<line x1="{xpt}" y1="{ypt}" x2="{xpt+u1*2.0}" y2="{ypt+v1*2.0}" stroke="#000000" stroke-width="0.0001"/>')
        for _tile_ in tile_to_rect:
            x0, y0, x1, y1 = tile_to_rect[_tile_]
            if rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y0), (x0, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x1, y0), (x1, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y0), (x1, y0)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y1), (x1, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y0), (x0, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x1, y0), (x1, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y0), (x1, y0)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y1), (x1, y1)): tiles_intersected_by_rays.add(_tile_)

svg.append('</svg>')

# num_of_tiles | total tiles | tiles intersected | xpt,   ypt
#           32 |        1024 |               338 | 0.48,  0.48
#           64 |        4096 |               711 | 0.48,  0.48
#           64 |        4096 |               562 | 0.728, 0.128
#           64 |        4096 |               446 | 0.044, 0.936
#           64 |        4096 |               625 | 0.465, 0.813
#           64 |        4096 |               548 | 0.716, 0.105
# /// just looking at the center tile & randomizing within there ///
# 64 | 4096 | 712 | 0.502,    0.509
# 64 | 4096 | 712 | 0.514,    0.509
# 64 | 4096 | 712 | 0.51,     0.505
# 64 | 4096 | 710 | 0.512,    0.511
# 64 | 4096 | 711 | 0.506,    0.513
# 64 | 4096 | 712 | 0.508006, 0.505877
# 64 | 4096 | 712 | 0.512125, 0.508032
# 64 | 4096 | 710 | 0.510063, 0.509533
# 64 | 4096 | 712 | 0.504302, 0.501294
# 64 | 4096 | 712 | 0.508994, 0.500143
# 64 | 4096 | 624 | 0.5,      0.5       # corner of the cell
# 64 | 4096 | 618 | 0.5,      0.515625  # corner of the cell
# 64 | 4096 | 621 | 0.515625, 0.5       # corner of the cell
# 64 | 4096 | 617 | 0.515625, 0.515625  # corner of the cell
# 64 | 4096 | 712 | 0.501,    0.5       # slight offset from the corner
# 64 | 4096 | 715 | 0.501,    0.501     # slight offset from the corner ### Highest Seen So Far
#
print(f'{num_of_tiles} | {len(tile_to_rect)} | {len(tiles_intersected_by_rays)} | {xpt}, {ypt}')

rt.tile([''.join(svg)])

In [ ]:
offtiles_intersected_by_rays = {(-14, -15),(18, 17),(19, -18),(-21, -54),(39, -40),(63, 25),(-12, 31),(8, 9),(0, 5),(40, 41),(19, 18),(-43, 43),(-24, 56),(-11, -5),(-19, -9),(-18, -44),(41, 42),(52, 51),(-15, -39),(16, -15),
(-18, -8),(-5, 11),(-26, -12),(-48, 49),(-28, 27),(-57, 0),(-4, 12),(0, -18),(-8, -22),(-16, 37),(31, -31),(-11, -28),(33, 15),(25, 11),(4, -12),(16, -38),(-27, 0),(-49, -19),(-57, -23),(-4, -11),
(14, 37),(-53, -53),(7, -2),(26, 11),(0, -41),(18, 7),(-55, 23),(-36, 36),(-56, -22),(-24, 10),(23, -58),(-52, -52),(-20, -20),(0, -5),(-8, -9),(11, 4),(12, -31),(-22, 56),(23, -22),(24, -57),
(43, -44),(21, 54),(-19, -19),(25, 24),(38, -37),(57, -24),(58, -59),(3, 5),(-46, 0),(-5, 1),(14, 14),(26, -12),(-22, -52),(-4, 2),(47, 47),(-26, 63),(0, -28),(-14, 37),(20, -50),(-35, 14),
(6, 15),(-24, 23),(-20, -7),(-29, 30),(-9, 8),(10, 21),(-18, -41),(-27, -10),(-15, -36),(13, 32),(46, -20),(-17, 40),(16, 37),(50, -50),(-5, 14),(36, 15),(-18, 44),(0, -51),(-54, -22),(30, -29),
(-24, 0),(17, 38),(29, 12),(21, 8),(-8, -19),(45, -44),(2, -6),(-46, -46),(14, -32),(20, 48),(-61, -26),(-9, 21),(-17, 17),(23, 53),(-57, -56),(3, -5),(22, 8),(-5, -9),(-56, -55),(-24, -23),
(-23, -58),(42, -41),(-12, -12),(29, -11),(0, -38),(60, -24),(6, 5),(-22, 23),(-43, 0),(-1, 1),(-2, 1),(-23, -22),(11, -29),(23, -55),(10, -25),(21, 21),(-13, 28),(25, -9),(39, 38),(10, 11),
(-17, -6),(57, -57),(2, 7),(56, -22),(-63, 27),(54, 54),(-32, 14),(-16, -42),(43, 44),(48, -21),(0, -61),(59, 25),(19, -48),(-15, 39),(-36, 16),(-24, -10),(9, 24),(-13, 5),(-62, 0),(44, -19),
(-10, 10),(-9, -25),(-44, 17),(-51, -22),(0, 60),(32, 12),(-17, 7),(49, -48),(-19, 46),(-58, -25),(-48, 19),(-7, 20),(-54, -55),(16, 40),(0, -48),(6, -5),(-21, -22),(-50, -49),(-32, -32),(0, 37),
(33, -15),(-10, 23),(-23, 53),(-17, -16),(2, -3),(-11, 27),(-40, 0),(20, 51),(13, 12),(23, 56),(16, 17),(17, -18),(-24, -56),(9, -22),(-6, -2),(-61, 62),(-14, -6),(46, 45),(-22, -10),(-3, 3),
(-23, -55),(49, 50),(-20, -50),(61, 24),(-13, -5),(-21, -9),(-2, 4),(-1, 4),(-29, -13),(-10, 0),(50, 51),(10, -22),(51, -21),(-17, -39),(56, -55),(62, 25),(54, 21),(0, 50),(-50, 49),(16, -6),
(-59, 0),(47, -19),(-15, 6),(18, -46),(-16, -39),(9, -9),(15, 34),(-14, 7),(-13, -28),(6, -15),(-49, 49),(-10, -23),(-45, 19),(-60, -24),(0, 27),(-29, 0),(-9, -22),(4, -3),(31, 14),(23, 10),
(24, -25),(-20, 48),(-58, -58),(24, 11),(16, 7),(36, -15),(-19, 49),(22, 50),(-26, 10),(-25, -25),(-6, -12),(5, -3),(-3, -7),(-43, -43),(21, -22),(-12, 30),(8, 8),(0, 4),(-56, 23),(-10, -10),
(-43, 42),(-24, 55),(0, 40),(41, 41),(-42, 43),(4, 10),(-23, 56),(23, 23),(16, -16),(1, 4),(-7, 0),(56, 56),(-37, 14),(9, -19),(-46, 45),(-4, 11),(7, 20),(-56, 0),(-16, 36),(8, 21),
(0, 17),(11, 26),(32, -31),(4, -13),(23, 0),(16, -39),(34, 15),(-49, -20),(17, -38),(28, -29),(-26, 0),(14, 36),(26, 10),(5, -13),(-56, -23),(15, 37),(-52, -53),(8, -2),(27, 11),(0, -6),
(18, 42),(19, 7),(25, -62),(-54, 23),(-35, 36),(39, -15),(12, -32),(-23, 10),(23, -23),(24, -58),(-63, -26),(-51, -52),(21, 53),(-19, -20),(-48, -47),(12, 4),(13, -31),(4, 0),(-18, -19),(-27, 12),
(14, 13),(-25, -58),(25, 59),(-15, -14),(-22, -53),(58, -24),(22, 53),(-4, 1),(15, 14),(27, -12),(18, 19),(-21, -52),(20, -51),(39, -38),(63, 27),(48, 47),(-20, -8),(54, -22),(0, 7),(-25, 63),
(-8, 3),(-23, 23),(23, -10),(63, 63),(35, -36),(-19, -7),(-18, -42),(-27, -11),(-15, -37),(56, 23),(-17, 39),(50, -51),(-5, 13),(-26, -10),(-34, -14),(26, 0),(-14, -36),(-28, 29),(30, -30),(0, -16),
(-8, -20),(-23, 0),(51, -51),(-11, -26),(4, -10),(-9, 20),(56, 0),(-17, 16),(16, -36),(3, -6),(-7, -20),(-56, -56),(-4, -9),(42, -42),(-12, -13),(7, 0),(0, -39),(-16, 16),(-23, -23),(23, -56),
(21, 20),(0, -3),(-8, -7),(12, -29),(-42, 0),(24, -55),(10, 10),(56, -23),(2, 6),(54, 53),(-32, 13),(25, 26),(37, 0),(-63, 62),(58, -57),(43, 43),(3, 7),(-5, 3),(26, -10),(59, 24),
(-22, -50),(-36, 15),(19, -49),(-16, -7),(-24, -11),(-4, 4),(9, 23),(-12, 0),(8, -22),(54, -55),(0, -26),(20, -48),(6, 17),(-23, -10),(-31, -14),(45, -19),(-9, 10),(10, 23),(-17, 6),(-38, -17),
(49, -49),(14, -7),(13, 34),(16, 39),(15, -6),(-18, 46),(28, 13),(-47, 19),(0, -49),(-16, 6),(17, 40),(-50, -50),(21, 10),(-61, -60),(-56, 55),(-10, 22),(10, 0),(2, -4),(34, -15),(20, 50),
(-61, -24),(-9, 23),(-49, -50),(23, 55),(3, -3),(14, 6),(61, -62),(-39, 0),(24, 56),(-16, -17),(17, 17),(-23, -56),(-20, -51),(40, 0),(0, -36),(-13, -6),(6, 7),(-2, 3),(-1, 3),(23, -53),
(50, 50),(10, -23),(56, -56),(-13, 30),(0, 49),(39, 40),(-9, 0),(-16, -40),(17, -6),(-58, 0),(48, -19),(0, -59),(-14, 6),(19, -46),(6, -16),(-49, 48),(-45, 18),(21, 0),(39, 17),(44, -17),
(-9, -23),(10, -10),(-44, 19),(-51, -20),(0, 62),(32, 14),(24, 10),(16, 6),(36, -16),(-19, 48),(15, -39),(-58, -23),(-48, 21),(-54, -53),(5, -4),(-43, -44),(16, 42),(17, 7),(63, -26),(-37, 36),
(9, 3),(-25, 10),(6, -3),(-21, -20),(-2, -7),(-42, -43),(-56, 22),(-10, -11),(-57, 57),(0, 39),(-29, 12),(12, 13),(-9, -10),(-42, 42),(-23, 55),(23, 22),(43, 0),(-11, 29),(-60, 24),(1, 3),
(13, 14),(45, 46),(24, 23),(56, 55),(17, -16),(9, -20),(-6, 0),(46, 47),(-22, -8),(5, 9),(-3, 5),(57, 56),(-23, -53),(-20, -48),(61, 26),(21, -10),(8, 20),(11, 25),(-29, -11),(54, 23),
(0, 52),(12, 26),(-50, 51),(24, 0),(-30, 29),(17, -39),(5, -14),(29, -29),(15, 36),(-25, 0),(27, 10),(6, -13),(18, 41),(25, -63),(39, -16),(-10, -21),(-63, -27),(54, 0),(0, 29),(20, 7),
(-48, -48),(13, -32),(24, -23),(35, -14),(-20, 50),(-27, 11),(-47, -47),(25, 58),(-15, -15),(13, 4),(-38, 38),(22, 52),(-26, 12),(-14, -14),(-21, -53),(-3, -5),(-57, 24),(-12, 32),(54, -23),(0, 6),
(19, 19),(-43, 44),(-24, 57),(-23, 22),(63, 62),(-11, -4),(-19, -8),(0, 42),(52, 52),(4, 12),(24, -10),(56, 22),(-38, 15),(36, -36),(-18, -7),(55, -23),(-26, -11),(-7, 2),(-34, -15),(53, 53),
(-14, -37),(57, 23),(49, 19),(-4, 13),(-25, -10),(-33, -14),(27, 0),(-16, 38),(31, -30),(51, -52),(42, 16),(0, 19),(11, 28),(4, -11),(-31, 31),(16, -37),(57, 0),(28, -27),(26, 12),(18, 8),
(-36, 37),(-16, 15),(15, 39),(-52, -51),(8, 0),(0, -4),(18, 44),(-41, -42),(-8, -8),(19, 9),(11, 5),(12, -30),(24, -56),(43, -43),(-51, -50),(-19, -18),(-40, -41),(25, 25),(13, -29),(4, 2),
(57, -23),(3, 6),(-18, -17),(14, 15),(-7, -8),(25, 61),(26, -11),(-22, -51),(38, 0),(-4, 3),(15, 16),(47, 48),(26, 25),(27, -10),(0, -27),(59, 59),(-21, -50),(20, -49),(-35, 15),(-24, 24),
(48, 49),(0, 9),(10, 22),(35, -34),(55, -56),(-15, -35),(46, -19),(-17, 41),(50, -49),(-5, 15),(15, -7),(-14, -34),(17, 39),(42, -17),(-12, 12),(29, 13),(0, -14),(-8, -18),(60, 0),(11, -5),
(-11, -24),(-20, 7),(-21, 48),(-61, -25),(-9, 22),(-17, 18),(3, -4),(22, 9),(14, 5),(-34, -35),(24, 55),(-4, -7),(-45, -45),(15, 6),(-12, -11),(62, -62),(7, 2),(0, -37),(11, -28),(23, -54),
(21, 22),(41, 0),(-13, 29),(57, -56),(2, 8),(54, 55),(3, 9),(-5, 5),(0, -60),(19, -47),(-24, -9),(9, 25),(54, -53),(0, -24),(-13, 6),(20, -46),(44, -18),(-10, 11),(-31, -12),(10, -11),
(-44, 18),(44, 18),(10, 25),(-17, 8),(-46, -19),(14, -5),(-48, 20),(16, 41),(17, 6),(7, -8),(0, -47),(17, 42),(-57, 56),(33, -14),(-22, 50),(-29, 11),(-56, 57),(-10, 24),(2, -2),(2, -1),
(-38, -38),(13, 13),(-49, -48),(23, 57),(-5, -5),(-60, 59),(-59, 24),(61, -60),(-28, 11),(46, 46),(-16, -15),(-23, -54),(28, 28),(-20, -49),(-14, 31),(-13, -4),(-21, -8),(-2, 5),(-1, 5),(10, -21),
(61, 61),(-19, -48),(62, 26),(54, 22),(0, 51),(14, -15),(-28, -12),(18, -45),(-36, -16),(-16, -38),(9, -8),(55, 22),(-12, -31),(29, -30),(-6, 12),(0, -57),(6, -14),(-49, 50),(-43, -19),(-45, 20),
(-9, -21),(-53, 52),(-47, -48),(16, 8),(-38, 37),(36, -14),(-26, 11),(27, -63),(5, -2),(5, -1),(-3, -6),(-43, -42),(-57, 23),(-37, 38),(9, 5),(21, -21),(-13, -14),(-2, -5),(-1, -5),(-56, 24),
(-10, -9),(-27, -62),(0, 41),(20, 19),(4, 11),(-23, 57),(23, 24),(24, -11),(36, -37),(-60, 26),(55, -24),(1, 5),(-48, 0),(53, 52),(24, 25),(56, 57),(-37, 15),(-46, 46),(-25, -11),(-6, 2),
(-33, -15),(-15, 33),(5, 11),(-3, 7),(57, 58),(58, 23),(8, 22),(0, 18),(33, -34),(11, 27),(-10, 4),(-31, 30),(0, 54),(-41, 16),(12, 28),(-30, 31),(28, -28),(-7, 14),(5, -12),(58, 0),
(9, -5),(15, 38),(-34, 33),(27, 12),(18, 43),(19, 8),(25, -61),(-63, -25),(-51, -51),(0, 31),(19, 44),(20, 9),(12, 5),(13, -30),(4, 1),(-18, -18),(1, -5),(25, 60),(-36, -36),(58, -23),
(22, 54),(-18, 18),(15, 15),(21, -54),(40, -41),(27, -11),(59, 58),(-21, -51),(-3, -3),(-45, 0),(48, 48),(54, -21),(0, 8),(-8, 4),(30, 30),(-24, 59),(-23, 24),(23, -9),(35, -35),(56, 24),
(-38, 17),(-27, 26),(-7, 4),(-34, -13),(-15, 0),(-14, -35),(57, 25),(49, 21),(42, -18),(-12, 11),(-13, -34),(0, -15),(-16, 40),(51, -50),(-11, -25),(0, 21),(-41, -17),(-8, 17),(12, -5),(4, -9),
(-50, 20),(43, -18),(-11, 11),(-19, 7),(47, -48),(-7, -19),(-4, -8),(-45, -46),(15, 5),(-34, 0),(62, -63),(-16, 17),(-44, -45),(-58, 57),(62, -27),(8, 2),(0, -1),(0, -2),(-41, -40),(12, -28),
(-11, -12),(-20, 19),(59, -60),(-63, 63),(3, 8),(22, 21),(-5, 4),(-16, -6),(-4, 5),(55, 54),(7, 14),(8, -21),(54, -54),(0, -25),(-21, -48),(20, -47),(-23, -9),(-31, -13),(-8, 7),(60, 25),
(52, 21),(44, 17),(45, -18),(10, 24),(-30, -12),(-38, -16),(-46, -20),(14, -6),(-61, 0),(-15, -33),(45, 18),(37, 14),(-17, 43),(-51, 50),(15, -5),(-47, 20),(18, 0),(-16, 7),(17, 41),(8, -8),
(0, -12),(-8, -16),(-56, 56),(-31, 0),(-38, -39),(-20, 9),(34, -14),(-21, 50),(-9, 24),(-49, -49),(3, -2),(3, -1),(-37, -38),(-5, -6),(14, 7),(61, -61),(-34, -33),(24, 57),(-16, -16),(17, 18),
(-4, -5),(-59, 59),(-58, 24),(28, 27),(7, 4),(0, -35),(11, -26),(30, -13),(29, 28),(61, 60),(-19, -49),(-13, 31),(-50, 0),(-44, 43),(62, 61),(-28, -13),(51, 51),(15, -15),(-12, -32),(7, -19),
(0, -58),(19, -45),(0, -22),(10, -9),(0, 63),(-52, 52),(15, -38),(-28, 0),(17, 8),(63, -25),(-37, 37),(9, 4),(-25, 11),(0, -45),(6, -2),(-2, -6),(-42, -42),(-57, 58),(-27, -63),(-22, 52),
(-29, 13),(-10, 26),(-9, -9),(10, 4),(2, 0),(22, -22),(-60, 25),(-26, -62),(24, 24),(-5, -3),(-47, 0),(5, 10),(-3, 6),(57, 57),(-25, 24),(21, -9),(-20, -47),(-14, 33),(52, -22),(-2, 7),
(58, 58),(-10, 3),(-45, 45),(0, 53),(34, -34),(12, 27),(-9, 4),(-17, 0),(-30, 30),(14, -13),(-40, 16),(13, 28),(18, -43),(-36, -14),(-16, -36),(29, -28),(47, 20),(-6, 14),(0, -55),(6, -12),
(-43, -17),(-33, 33),(0, 30),(19, 43),(20, 8),(25, -26),(-9, -19),(-20, 51),(14, -36),(26, -62),(-47, -46),(13, 5),(-36, -37),(-38, 39),(-18, 17),(-14, -13),(5, 0),(-35, -36),(-3, -4),(-57, 25),
(41, -41),(-13, -12),(19, 20),(-2, -3),(-1, -3),(30, 29),(-24, 58),(-44, 0),(22, -55),(0, 43),(20, 21),(52, 53),(31, 30),(4, 13),(24, -9),(35, 0),(-38, 16),(36, -35),(55, -22),(-7, 3),
(53, 54),(57, 24),(49, 20),(-26, 26),(-25, -9),(-33, -13),(-15, 35),(-14, 0),(5, 13),(-3, 9),(-16, 39),(58, 25),(50, 21),(42, 17),(0, 20),(-41, -18),(-8, 16),(11, 29),(12, -6),(-63, 0),
(-31, 32),(43, -19),(-11, 10),(-40, -17),(-48, -21),(12, 30),(13, -5),(16, 0),(-18, 7),(48, -48),(-7, 16),(5, -10),(-62, -63),(-49, 19),(-33, 0),(-41, -41),(18, 45),(-22, 9),(25, -59),(-29, -30),
(-53, 21),(43, -42),(0, 33),(-40, -40),(13, -28),(4, 3),(-19, 19),(1, -3),(-7, -7),(25, 62),(37, 36),(-24, -60),(22, 56),(21, -52),(40, -39),(26, 26),(59, 60),(-21, -49),(38, 37),(-24, 25),
(27, 27),(0, 10),(60, 24),(-24, 61),(-17, -43),(-30, -13),(-11, 0),(55, -55),(60, 60),(53, 21),(-15, -34),(46, -18),(-17, 42),(-37, -16),(-27, 28),(-14, -33),(46, 18),(42, -16),(-12, 13),(8, -9),
(0, -13),(19, 0),(-8, -17),(11, -4),(-16, 42),(-20, 8),(0, 23),(-8, 19),(-21, 49),(4, -7),(-50, 22),(-30, 0),(-19, 9),(22, 10),(47, -46),(-7, -17),(-34, -34),(49, 0),(-45, -44),(-59, 58),
(-58, 23),(15, 7),(61, -26),(7, 3),(-33, -33),(62, -61),(11, -27),(-44, -43),(-58, 59),(-12, 26),(62, -25),(8, 4),(0, 0),(-8, -4),(-62, 25),(12, -26),(31, -13),(-31, 12),(-11, -10),(-20, 21),
(33, 33),(51, 50),(3, 10),(22, 23),(-5, 6),(15, -16),(7, -20),(50, -22),(-4, 7),(55, 56),(7, 16),(8, -19),(0, -23),(20, -45),(44, 19),(10, 26),(-52, 51),(22, 0),(-46, -18),(37, 16),
(-51, 52),(14, 32),(7, -7),(0, -46),(17, 43),(-60, -61),(0, -10),(33, -13),(-22, 51),(52, 0),(-10, 25),(10, 3),(-38, -37),(21, 49),(22, -23),(-27, -28),(-26, -63),(-21, 52),(23, 58),(3, 0),
(-37, -36),(-5, -4),(-60, 60),(-59, 25),(-25, -62),(-28, 12),(24, 59),(-4, -3),(28, 29),(0, -33),(19, -20),(-14, 32),(11, -24),(-2, 6),(58, 57),(29, 30),(-45, 44),(-25, 59),(61, 62),(-19, -47),
(62, 27),(-13, 33),(33, 0),(34, -35),(53, -22),(-9, 3),(-44, 45),(62, 63),(22, -10),(14, -14),(-28, -11),(18, -44),(-36, -15),(-16, -37),(49, -20),(-5, 9),(55, 23),(-12, -30),(-39, 16),(47, 19),
(7, -17),(-48, 47),(-6, 13),(0, -56),(18, -8),(19, -43),(-35, -14),(-43, -18),(63, 0),(48, 20),(40, 16),(-33, 32),(-42, -17),(-50, -21),(-9, -20),(-53, 53),(14, -37),(25, 9),(26, -63),(3, -10),
(-5, -14),(55, 0),(15, -36),(26, -27),(27, -62),(21, -20),(0, -43),(41, -42),(-13, -13),(6, 0),(-2, -4),(-1, -4),(-31, -31),(22, -56),(-22, 54),(20, 20),(-23, 58),(2, 2),(-41, 40),(35, 35),
(36, 0),(-37, 16),(-46, 47),(-26, 25),(-6, 3),(-15, 34),(5, 12),(-3, 8),(58, 24),(9, 19),(50, 20),(-26, 61),(-25, 26),(-20, -45),(-14, 35),(-13, 0),(33, -33),(6, 13),(-10, 5),(-32, -14),
(0, 55),(-41, 17),(12, 29),(13, -6),(51, 20),(-39, -17),(48, -49),(-7, 15),(13, 30),(18, -41),(5, -11),(17, 0),(9, -4),(-18, 42),(-6, 16),(40, -17),(-34, 34),(-22, 8),(25, -60),(0, 32),
(19, 45),(-21, 9),(25, -24),(45, -46),(12, 6),(2, -8),(-52, 21),(14, -34),(-19, 18),(1, -4),(-39, -40),(26, -60),(-36, -35),(-24, -61),(-18, 19),(22, 55),(21, -53),(-6, -7),(40, -40),(5, 2),
(-3, -2),(-3, -1),(9, 9),(26, 61),(27, 26),(-22, 21),(39, 0),(-10, -5),(30, 31),(-24, 60),(10, -27),(-17, -44),(42, 42),(22, -53),(0, 45),(60, 59),(31, 32),(-27, 27),(-6, 6),(-13, -33),
(52, -53),(-16, 41),(0, 22),(-41, -16),(20, 0),(-8, 18),(12, -4),(4, -8),(-50, 21),(43, -17),(-11, 12),(-19, 8),(47, -47),(-48, -19),(-7, -18),(-7, 18),(-6, -17),(-33, -34),(-15, 14),(-62, -61),
(-49, 21),(50, 0),(-44, -44),(-58, 58),(62, -26),(8, 3),(25, -57),(-29, -28),(12, -27),(31, -14),(-11, -11),(-20, 20),(0, 35),(33, 32),(59, -59),(32, -13),(4, 5),(-30, 12),(-11, 25),(22, 22),
(34, 33),(-61, 24),(37, 38),(9, -24),(55, 55),(21, -50),(-34, 14),(7, 15),(8, -20),(-53, 0),(8, 16),(0, 12),(-8, 8),(60, 26),(52, 22),(-38, -15),(53, 23),(45, 19),(37, 15),(-17, 44),
(17, -43),(-37, -14),(-51, 51),(14, 31),(-14, -31),(46, 20),(38, 16),(-16, 8),(-45, -19),(-53, -23),(8, -7),(0, -11),(21, 48),(-60, -26),(-8, 21),(34, -13),(-21, 51),(53, 0),(4, -5),(-9, 25),
(-37, -37),(-25, -63),(-7, -15),(24, 58),(-4, -4),(-59, 60),(-58, 25),(61, -24),(-39, 38),(0, -34),(11, -25),(30, -12),(-12, 28),(29, 29),(-25, 58),(0, 2),(20, -20),(-13, 32),(-8, -2),(-62, 27),
(53, -23),(-31, 14),(-44, 44),(62, 62),(34, 0),(44, 44),(51, 52),(49, -21),(15, -14),(-39, 15),(7, -18),(19, -44),(-35, -15),(50, -20),(-4, 9),(-12, 5),(48, 19),(8, -17),(-47, 47),(0, -21),
(19, -8),(11, -12),(-43, 17),(-42, -18),(-50, -22),(-20, 0),(41, 16),(16, -41),(-52, 53),(-5, -15),(-32, 31),(15, -37),(38, -17),(27, -27),(0, -44),(-42, -41),(-31, -32),(-60, -59),(-54, 21),(-22, 53),
(-10, 27),(-9, -8),(24, -60),(-30, -31),(10, 5),(2, 1),(22, -21),(-27, -26),(-26, -61),(-21, 54),(3, 2),(35, 34),(-5, -2),(-5, -1),(-40, 40),(18, -19),(-22, -55),(24, 61),(36, 35),(-12, -5),
(-26, 60),(-25, 25),(-20, -46),(21, -8),(0, -31),(-14, 34),(6, 12),(52, -21),(-2, 8),(58, 59),(-45, 46),(-25, 61),(-19, -45),(34, -33),(-9, 5),(22, -8),(-40, 17),(13, 29),(18, -42),(-18, 41),
(-12, -28),(7, -15),(-6, 15),(0, -54),(-33, 34),(41, -17),(-13, 12),(-21, 8),(25, -25),(10, -5),(14, -35),(20, 45),(26, -61),(13, 6),(46, -46),(37, -15),(3, -8),(-51, 21),(-5, -12),(15, -34),
(26, -25),(-36, 0),(-35, -35),(5, 1),(9, 8),(41, -40),(26, 60),(6, 2),(-2, -1),(-1, -1),(-2, -2),(-1, -2),(42, 41),(22, -54),(0, 44),(-21, 21),(31, 31),(37, -38),(-9, -5),(2, 4),
(-41, 42),(32, 32),(-27, 62),(-26, 27),(-6, 5),(-15, 36),(-55, 0),(5, 14),(-3, 10),(9, 21),(50, 22),(42, 18),(53, -53),(-32, -12),(0, 57),(-40, -16),(-48, -20),(12, 31),(13, -4),(51, 22),
(43, 18),(35, 14),(-19, 43),(-18, 8),(-39, -15),(48, -47),(-7, 17),(-47, -19),(-55, -23),(5, -9),(-62, -62),(-49, 20),(28, 11),(-14, 14),(-22, 10),(-62, -26),(25, -58),(18, 46),(-29, -29),(44, -45),
(-53, 22),(0, 34),(-40, -39),(19, 47),(-61, -62),(4, 4),(32, -14),(-11, 24),(-19, 20),(1, -2),(1, -1),(-39, -38),(-7, -6),(-28, -29),(59, -24),(25, 63),(37, 37),(-24, -59),(9, -25),(28, -12),
(21, -51),(-34, 13),(-6, -5),(26, 27),(60, -60),(5, 4),(38, 38),(-3, 0),(-33, 14),(26, 63),(27, 28),(0, 11),(-10, -3),(-17, -42),(-52, 0),(22, -51),(0, 47),(55, -54),(60, 61),(53, 22),
(13, -14),(17, -44),(-37, -15),(-7, 7),(-14, -32),(46, 19),(38, 15),(17, -8),(-45, -20),(-13, -31),(-22, 0),(52, -51),(-10, -26),(-44, -19),(0, 24),(-54, 53),(-8, 20),(39, 15),(-20, 45),(-7, -16),
(-55, -56),(61, -25),(-6, -15),(-33, -32),(-15, 16),(5, -6),(-22, -23),(-3, -10),(-12, 27),(0, 1),(-8, -3),(20, -21),(11, 10),(-62, 26),(31, -12),(-31, 13),(33, 34),(-62, 62),(12, 11),(44, 43),
(4, 7),(1, 1),(-7, -3),(34, 35),(-61, 26),(-15, -7),(45, 44),(-49, 0),(50, -21),(-4, 8),(-12, 4),(21, -48),(7, 17),(8, -18),(-47, 46),(19, -9),(30, 0),(8, 18),(0, 14),(20, -8),
(12, -12),(-42, 17),(10, 27),(16, -42),(-11, 4),(-19, 0),(17, -41),(-46, 19),(14, 33),(7, -6),(27, -28),(17, 44),(-53, -21),(-60, -60),(0, -9),(-55, 55),(11, 0),(30, 13),(24, -61),(21, 50),
(-59, -59),(-27, -27),(-21, 53),(13, -34),(-38, 0),(3, 1),(-60, 61),(-26, -26),(-25, -61),(-40, 39),(-28, 13),(-22, -56),(24, 60),(-4, -2),(-4, -1),(-12, -6),(-39, 40),(7, 7),(0, -32),(19, -19),
(-25, 60),(-19, -46),(40, 40),(-8, 0),(-13, 34),(53, -21),(22, -9),(-18, -45),(49, -19),(-5, 10),(55, 24),(-12, -29),(-39, 17),(7, -16),(-48, 48),(18, -7),(-16, 0),(48, 21),(40, 17),(0, -19),
(41, -18),(11, -10),(-43, 19),(31, -32),(-42, -16),(-50, -20),(-11, -29),(-53, 54),(41, 18),(33, 14),(25, 10),(46, -47),(37, -16),(3, -9),(-51, 20),(-5, -13),(-32, 33),(14, 0),(15, -35),(26, -26),
(38, -15),(-57, -24),(-4, -12),(-53, -54),(7, -3),(0, -42),(-55, 22),(-36, 35),(-35, 0),(-24, 9),(-63, -62),(-31, -30),(-20, -21),(-22, 55),(-21, 20),(44, 0),(-30, -29),(2, 3),(-41, 41),(32, 31),
(38, -38),(3, 4),(-5, 0),(35, 36),(18, -17),(36, 37),(9, 20),(47, 46),(-26, 62),(0, -29),(-14, 36),(-54, 0),(33, -32),(6, 14),(53, -54),(-19, -43),(-32, -13),(0, 56),(-41, 18),(25, 0),
(-29, 29),(51, 21),(43, 17),(-39, -16),(-47, -20),(13, 31),(-55, -24),(16, 36),(36, 14),(9, -3),(-18, 43),(-12, -26),(-34, 35),(-6, 17),(0, -52),(-14, 13),(-54, -23),(40, -16),(-62, -27),(29, 11),
(-13, 14),(19, 46),(-21, 10),(45, -45),(10, -3),(2, -7),(-52, 22),(-46, -47),(-32, 0),(14, -33),(-39, -39),(20, 47),(59, -25),(-57, -57),(-5, -10),(28, -13),(47, 0),(-6, -6),(60, -61),(5, 3),
(-24, -24),(9, 10),(29, -12),(-33, 13),(26, 62),(60, -25),(-22, 22),(-1, 0),(-2, 0),(-10, -4),(10, -26),(42, 43),(22, -52),(27, 63),(0, 46),(25, -10),(-9, -3),(37, -36),(-63, 26),(-17, -7),
(57, -58),(-51, 0),(-7, 6),(28, 0),(-6, 7),(0, -62),(-15, 38),(-13, -32),(52, -52),(-10, -27),(-13, 4),(-21, 0),(-10, 9),(0, 59),(-19, 45),(-7, 19),(-6, -16),(-15, 15),(9, 0),(6, -6),
(44, -43),(-32, -33),(0, 36),(19, 49),(-62, 61),(59, -58),(32, -12),(-17, -17),(-30, 13),(-11, 26),(1, 0),(34, 34),(-7, -4),(-61, 25),(-28, -27),(16, 16),(-24, -57),(9, -23),(21, -49),(-6, -3),
(-34, 15),(-61, 61),(-14, -7),(5, 6),(-3, 2),(49, 49),(8, 17),(0, 13),(-21, -10),(20, -9),(-8, 9),(12, -13),(31, 0),(-42, 16),(51, -22),(-17, -40),(13, -12),(45, 20),(16, -7),(17, -42),
(-18, 0),(-46, 18),(47, -20),(-15, 5),(38, 17),(9, -10),(-45, -18),(15, 33),(-53, -22),(61, 0),(-13, -29),(-55, 54),(30, 12),(-10, -24),(-44, -17),(-52, -21),(-60, -25),(-59, -60),(0, 26),(-54, 55),
(-8, 22),(12, 0),(31, 13),(4, -4),(23, 9),(-20, 47),(-59, -24),(-58, -59),(-26, -27),(-7, -14),(-55, -54),(63, -62),(-37, 0),(-25, -26),(-6, -13),(7, 6),(-39, 39),(-22, -21),(-3, -8),(42, 0),
(-12, 29),(8, 7),(40, 39),(0, 3),(20, -19),(11, 12),(-18, -46),(41, 40),(4, 9),(44, 45),(16, -17),(-15, -5),(-4, 10),(-12, 6),(7, 19),(8, -16),(-47, 48),(0, -20),(19, -7),(11, -11),
(-43, 18),(0, 16),(41, 17),(33, 13),(32, -32),(16, -40),(-32, 32),(34, 14),(-49, -21),(38, -16),(-57, -25),(-4, -13),(14, 35),(15, 0),(7, -4),(27, -26),(-56, -24),(-63, -63),(8, -3),(0, -7),
(-54, 22),(-35, 35),(-23, 9),(23, -24),(24, -59),(-30, -30),(21, 52),(-26, -60),(45, 0),(38, -39),(3, 3),(-25, -59),(-40, 41),(18, -18),(-22, -54),(58, -25),(36, 36),(-4, 0),(-12, -4),(0, -30),
(18, 18),(39, -39),(63, 26),(-25, 62),(-20, -9),(-19, -44),(-8, 2),(-29, 28),(-18, -43),(-27, -12),(-15, -38),(-17, 38),(-5, 12),(-12, -27),(7, -14),(-28, 28),(0, -53),(30, -31),(0, -17),(41, -16),
(-13, 13),(-8, -21),(10, -4),(-11, -27),(20, 46),(46, -45),(-9, 19),(37, -14),(-57, -58),(3, -7),(-51, 22),(-5, -11),(15, -33),(-35, -34),(-56, -57),(-24, -25),(-4, -10),(42, -43),(-53, -52),(29, -13),
(48, 0),(0, -40),(-55, 24),(60, -26),(6, 3),(-24, 11),(-23, -24),(23, -57),(-20, -19),(27, 62),(-21, 22),(25, -11),(-9, -4),(37, -37),(10, 9),(-17, -8),(56, -24),(2, 5),(-63, 25),(-32, 12),
(32, 33),(58, -58),(43, 42),(-5, 2),(-27, 63),(0, -63),(-15, 37),(-36, 14),(5, 15),(-16, -8),(9, 22),(29, 0),(6, 16),(53, -52),(0, 58),(12, 32),(45, -20),(-9, 9),(43, 19),(49, -50),
(35, 15),(-19, 44),(13, 33),(59, 0),(-55, -22),(16, 38),(36, 16),(-18, 45),(28, 12),(0, -50),(-14, 15),(-54, -21),(6, -7),(-62, -25),(44, -44),(-50, -51),(21, 9),(-53, 23),(19, 48),(-61, -61),
(-10, 21),(51, 0),(-17, -18),(2, -5),(-46, -45),(14, -31),(20, 49),(-28, -28),(23, 54),(16, 15),(-24, -58),(28, -11),(-61, 60),(60, -59),(5, 5),(-3, 1),(38, 39),(17, 16),(-23, -57),(49, 48),
(-33, 15),(6, 6),(-1, 2),(-2, 2),(50, 49),(10, -24),(-17, -41),(56, -57),(22, -50),(0, 48),(39, 39),(13, -13),(32, 0),(16, -8),(-7, 8),(-16, -41),(17, -7),(48, -20),(-14, 5),(-13, -30),
(6, -17),(-10, -25),(-44, -18),(-52, -22),(62, 0),(0, 25),(-54, 54),(39, 16),(31, 12),(-9, -24),(-51, -21),(-20, 46),(-59, -25),(0, 61),(13, 0),(-55, -55),(32, 13),(24, 9),(63, -63),(-19, 47),
(-58, -24),(-6, -14),(-54, -54),(5, -5),(-22, -22),(-3, -9),(63, -27),(-25, 9),(-21, -21),(-2, -8),(11, 11),(-32, -31),(0, 38),(-41, 0),(-62, 63),(12, 12),(-42, 41),(4, 8),(-23, 54),(-11, 28),
(1, 2),(-7, -2),(-15, -6),(45, 45),(17, -17),(-24, -55),(9, -21),(7, 18),(-14, -5),(-22, -9),(-3, 4),(61, 25),(8, 19),(0, 15),(20, -7),(-29, -12),(11, 24),(12, -11),(-42, 18),(32, -33),
(51, -20),(-17, -38),(-11, 5),(-60, 0),(34, 13),(-50, 50),(17, -40),(-46, 20),(14, 34),(-15, 7),(5, -15),(15, 35),(8, -4),(0, -8),(-55, 56),(-35, 34),(39, -17),(-10, -22),(21, 51),(-59, -58),
(0, 28),(-48, -49),(13, -33),(4, -2),(4, -1),(24, -24),(35, -15),(-20, 49),(-58, -57),(-27, 10),(-26, -25),(-25, -60),(-15, -16),(46, 0),(22, 51),(-25, -24),(7, 8),}
len(offtiles_intersected_by_rays)

In [ ]:
tile_to_rect = {}
for xi in range(num_of_tiles):
    x0, x1 = xi/float(num_of_tiles), (xi+1)/float(num_of_tiles)
    for yi in range(num_of_tiles):
        y0, y1 = yi/float(num_of_tiles), (yi+1)/float(num_of_tiles)
        tile_to_rect[(xi,yi)] = (x0, y0, x1, y1)
for _iters_ in range(10):
    xpt, ypt = random.random(), random.random()
    xi,  yi  = int(xpt*num_of_tiles), int(ypt*num_of_tiles)
    for _sector_ in range(16):
        a0, a1         = _sector_*2*pi/16, (_sector_+1)*2*pi/16
        u0, v0, u1, v1 = cos(a0), sin(a0), cos(a1), sin(a1)
        for _tile_ in tile_to_rect:
            x0, y0, x1, y1 = tile_to_rect[_tile_]
            if rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y0), (x0, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x1, y0), (x1, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y0), (x1, y0)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y1), (x1, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y0), (x0, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x1, y0), (x1, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y0), (x1, y0)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y1), (x1, y1)): offtiles_intersected_by_rays.add((_tile_[0]-xi, _tile_[1]-yi))

# @100 Iterations Per Run : 2092 -> 2188 -> 2213 -> 2222 -> 2237 -> 2256
# then (on top... 1000 iterations)  ... -> 2307 -> 2321 -> 2327 -> 2332 -> 2352 -> ... (2354) -> 2357
# ... final number is 2357 for 64 tiles ... but maybe it's actually 2361 if I had done this more methodically
print(len(offtiles_intersected_by_rays)) # 100 Iterations takes 19.3 seconds | 1000 Iterations takes 3m12s | 10K Iterations takes 67m54s | M1 Pro 16G 

i = 0
for x in offtiles_intersected_by_rays:
    print(x,end=',')
    i += 1
    if i % 20 == 0: print()

In [ ]:
num_of_tiles = 32
_iota_       = 10e-6

tile_to_rect = {}
for xi in range(num_of_tiles):
    x0, x1 = xi/float(num_of_tiles), (xi+1)/float(num_of_tiles)
    for yi in range(num_of_tiles):
        y0, y1 = yi/float(num_of_tiles), (yi+1)/float(num_of_tiles)
        tile_to_rect[(xi,yi)] = (x0, y0, x1, y1)
offtiles_intersected_by_rays = set()
for _tile_ in [(0,0), (0, num_of_tiles-1), (num_of_tiles-1, num_of_tiles-1), (num_of_tiles-1, 0)]:
    xi,  yi  = _tile_
    x0,  y0, x1, y1 = tile_to_rect[_tile_]
    xm,  ym         = (x0+x1)/2.0, (y0+y1)/2.0
    _positions_ = [(x0+_iota_, y0+_iota_),(xm, y0+_iota_),(x1-_iota_, y0+_iota_),
                   (x0+_iota_, ym),       (xm, ym),       (x1-_iota_, ym),
                   (x0+_iota_, y1-_iota_),(xm, y1-_iota_),(x1-_iota_, y1-_iota_)]
    for _position_ in _positions_:
        xpt, ypt = _position_
        for _sector_ in range(16):
            a0, a1         = _sector_*2*pi/16, (_sector_+1)*2*pi/16
            u0, v0, u1, v1 = cos(a0), sin(a0), cos(a1), sin(a1)
            for _tile_ in tile_to_rect:
                x0, y0, x1, y1 = tile_to_rect[_tile_]
                if rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y0), (x0, y1)) or \
                   rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x1, y0), (x1, y1)) or \
                   rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y0), (x1, y0)) or \
                   rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y1), (x1, y1)) or \
                   rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y0), (x0, y1)) or \
                   rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x1, y0), (x1, y1)) or \
                   rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y0), (x1, y0)) or \
                   rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y1), (x1, y1)): offtiles_intersected_by_rays.add((_tile_[0]-xi, _tile_[1]-yi))

xi_min, xi_max, yi_min, yi_max = 0, 0, 0, 0 # xi_min=-63 yi_min=-63 xi_max=63 yi_max=63 // for num_of_tiles = 64
for _xyi_ in offtiles_intersected_by_rays:
    xi, yi = _xyi_
    xi_min, xi_max = min(xi, xi_min), max(xi, xi_max)
    yi_min, yi_max = min(yi, yi_min), max(yi, yi_max)
xiyi_to_sector, sectors_seen = {}, set()
for xi in range(xi_min, xi_max+1):
    for yi in range(yi_min, yi_max+1):
        if (xi,yi) in offtiles_intersected_by_rays: continue
        xm, ym   = xi, yi
        _angle_  = atan2(ym, xm)
        _sector_ = int(_angle_*16.0/2.0/pi) # calc doesn't line up w/ current implementation
        xiyi_to_sector[(xi,yi)] = _sector_
        sectors_seen.add(_sector_)
print(f'{xi_min=} {yi_min=} {xi_max=} {yi_max=} | {len(offtiles_intersected_by_rays)=} | {len(xiyi_to_sector)=} | {len(sectors_seen)=}')
print(sectors_seen)
svg = [f'<svg x="0" y="0" width="512" height="512" viewBox="0 0 {num_of_tiles*2+2} {num_of_tiles*2+2}">']
svg.append(f'<rect x="0" y="0" width="{num_of_tiles*2+2}" height="{num_of_tiles*2+2}" rx="0.05" fill="#ffffff" />')
for _xyi_ in offtiles_intersected_by_rays:
    xi, yi = _xyi_
    svg.append(f'<rect x="{xi+(num_of_tiles+1)}" y="{yi+num_of_tiles+1}" width="1" height="1" rx="0.05" fill="#000000" />')
for _xyi_ in xiyi_to_sector:
    xi, yi   = _xyi_
    _sector_ = xiyi_to_sector[_xyi_]
    _color_  = rt.co_mgr.getColor(_sector_)
    svg.append(f'<rect x="{xi+(num_of_tiles+1)}" y="{yi+num_of_tiles+1}" width="1" height="1" rx="0.05" fill="{_color_}" />')
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
num_of_pts   = [10, 20, 5]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])
df = pl.DataFrame({'x':_xvals_, 'y':_yvals_, 'w':_weights_, 'c':_colors_}).with_row_index('__index__')
df_weight_sum = df['w'].sum()
# vvv -- PERFORMANCE ISSUES
# Explode
df = df.with_columns(pl.struct(['x','y','__index__']).implode().alias('_implode_')) \
        .explode('_implode_')                                            \
        .with_columns(pl.col('_implode_').struct.field('x')        .alias('_xo_'),
                        pl.col('_implode_').struct.field('y')        .alias('_yo_'),
                        pl.col('_implode_').struct.field('__index__').alias('_indexo_'))
df = df.filter(pl.col('__index__') != pl.col('_indexo_')) # don't compare the point with itself
# Sector Determination (per point basis)
_dx_ = pl.col('_xo_') - pl.col('x')
_dy_ = pl.col('_yo_') - pl.col('y')
df   = df.with_columns(((16*(pl.arctan2(_dy_, _dx_) + pl.lit(pi))/(pl.lit(2*pi))).cast(pl.Int64)).alias('sector'))
# Sector Summation
df   = df.group_by(['__index__','x','y','sector']).agg((pl.col('w').sum()).alias('_w_sum_'), (pl.col('w').sum() / df_weight_sum).alias('_w_ratio_'))
# ^^^ -- PERFORMANCE ISSUES
df